In [2]:
from datasets import load_from_disk
from disent_helper import simplify_nq_dataset
import pandas as pd

json = pd.read_json("train_nqa.jsonl", lines=True)
print(len(json))

307373


In [3]:
json.head()

,document_text,long_answer_candidates,question_text,annotations,document_url,example_id
0,Email marketing - Wikipedia <H1> Email marketi...,"[{'start_token': 14, 'top_level': True, 'end_t...",which is the most common use of opt-in e-mail ...,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Em...,5655493461695504401
1,The Mother ( How I Met Your Mother ) - wikiped...,"[{'start_token': 28, 'top_level': True, 'end_t...",how i.met your mother who is the mother,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Th...,5328212470870865242
2,Human fertilization - wikipedia <H1> Human fer...,"[{'start_token': 14, 'top_level': True, 'end_t...",what type of fertilisation takes place in humans,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Hu...,4435104480114867852
3,List of National Football League career quarte...,"[{'start_token': 28, 'top_level': True, 'end_t...",who had the most wins in the nfl,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Li...,5289242154789678439
4,Roanoke Colony - wikipedia <H1> Roanoke Colony...,"[{'start_token': 32, 'top_level': True, 'end_t...",what happened to the lost settlement of roanoke,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Ro...,5489863933082811018


In [74]:
import pandas as pd
import numpy as np
from nqa_text_util import simplify_nq_example


# Correct version
def new_simplify_nq_dataset( json ):
    questions = []
    answers = []
    contexts = []

    # extracting factual answers
    for i, j in json.iterrows():
        #j = simplify_nq_example(j) # train set already simplified
        question = j['question_text']
        annots = j['annotations']
        context = ''
        answer = ''

        sub_contexts = []
        sub_answers = []
            
        for a in annots:
            long_s = a['long_answer']['start_token']
            long_e = a['long_answer']['end_token']
            context = ' '.join( j['document_text'].split()[long_s:long_e] )
            if context == '':
                continue

            if '<Table>' in context:
                continue

            if a['short_answers']:
                answer = ''
                for rng in a['short_answers']:        
                    short_s = rng['start_token']
                    short_e = rng['end_token']
                    answer = ' '.join( j['document_text'].split()[short_s:short_e] )                    
            else:
                sub_contexts.append(context)
                sub_answers.append('')


        # for i in range(len(sub_contexts)):
        #     #if sub_answers[i] == '':                    
        #     print(question)
        #     print(sub_contexts[i])
        #     print(f'\tAnswer: [{sub_answers[i]}]')

        # if answer:
        
        if len(sub_contexts) > 0:
            idx = np.argmin([ len(ctx) for ctx in sub_contexts ])
                
            questions.append(question)
            contexts.append(sub_contexts[idx])
            answers.append(sub_answers[idx])

    df = pd.DataFrame( list(zip(questions, contexts, answers)), columns=['question', 'context', 'answer'] )
    return df


In [75]:
df = new_simplify_nq_dataset(json)
#df.to_csv('dev_nqa2.csv')
df.head()

,question,context,answer
0,what type of fertilisation takes place in humans,<P> The process of fertilization involves a sp...,
1,what indian tribe did the acadians form friend...,<P> The survival of the Acadian settlements wa...,
2,what is the use of jdk in java,<P> The Java Development Kit ( JDK ) is an imp...,
3,where is a georgia concealed carry permit valid,<P> Georgia reciprocates in recognizing firear...,
4,when were the plus and minus signs first recorded,<P> In Europe in the early 15th century the le...,


In [93]:
#df.to_csv('unans_train_set.csv')
#df_f.columns.values


new_df = pd.DataFrame( {'question': df['question'].values, 'context': df['context'].values, 'parametric_answer': 'unanswerable', 
              'contextual_answer': 'unanswerable', 'answerable': False, 'type': 'unanswerable', 'input': ('question: ' + df['question'] + '\ncontext: ' + df['context'] ), 'output' : '' } )

new_df.to_csv('unanswerable_train_set.csv')
new_df.head()


,question,context,parametric_answer,contextual_answer,answerable,type,input
0,what type of fertilisation takes place in humans,<P> The process of fertilization involves a sp...,unanswerable,unanswerable,False,unanswerable,question: what type of fertilisation takes pla...
1,what indian tribe did the acadians form friend...,<P> The survival of the Acadian settlements wa...,unanswerable,unanswerable,False,unanswerable,question: what indian tribe did the acadians f...
2,what is the use of jdk in java,<P> The Java Development Kit ( JDK ) is an imp...,unanswerable,unanswerable,False,unanswerable,question: what is the use of jdk in java\ncont...
3,where is a georgia concealed carry permit valid,<P> Georgia reciprocates in recognizing firear...,unanswerable,unanswerable,False,unanswerable,question: where is a georgia concealed carry p...
4,when were the plus and minus signs first recorded,<P> In Europe in the early 15th century the le...,unanswerable,unanswerable,False,unanswerable,question: when were the plus and minus signs f...


In [107]:
# Concatenate the two dataframes vertically
concatenated_df = pd.concat([new_df, train_df], axis=0)

# Randomize the entries in the dataframe
randomized_df = concatenated_df.sample(frac=1, random_state=155)
randomized_df.reset_index(drop=True, inplace=True)

len(randomized_df)

290046

In [128]:
randomized_df.sample(1)

,question,context,parametric_answer,contextual_answer,answerable,type,input,output
27247,do non alcoholic drinks have alcohol in them,<P> However alcoholic beverages can not be fur...,unanswerable,unanswerable,False,unanswerable,question: do non alcoholic drinks have alcohol...,NaN


In [129]:
randomized_df.to_csv('new_train_set.csv')

In [130]:
import tarfile
import csv
import gzip

with tarfile.open('test_sets.csv.tar.gz', 'r:gz') as tar:
    for member in tar.getmembers():
        if member.name.endswith('.csv'):
            csv_file = tar.extractfile(member)
            break

    test_df = pd.read_csv(csv_file)
    tar.close()

df_f = test_df[test_df['type'] == 'factual'].reset_index()
df_f.drop('Unnamed: 0', axis=1, inplace=True)
df_f.drop('index', axis=1, inplace=True)



with tarfile.open('(s) f+a - train.csv.tar.gz', 'r:gz') as tar:
    for member in tar.getmembers():
        if member.name.endswith('.csv'):
            csv_file = tar.extractfile(member)
            break

    train_df = pd.read_csv(csv_file)
    tar.close()

train_df.drop('Unnamed: 0', axis=1, inplace=True)
train_df.drop('Unnamed: 0.1', axis=1, inplace=True)

len(train_df)


256620